In [2]:
from simplelhs import LatinHypercubeSampling
from simplelhs import unnormalise
import numpy as np
import pandas as pd
import random
import time
import sys

random.seed(time.time())

In [13]:
names = ['A','D','S','T','p_hi','p_lo','b_prob','ps','ews','lambda','t_r','peer_it']
lower = np.array([0.0, 1.0, 0.0, 1.0, 0.3, 0.01, 0.2, 0.0, 0.0, 0.0, 1, 1])
upper = np.array([100.0, 10.0, 1.0, 10.0, 1.0, 0.1, 0.4, 1.0, 1.0, 1.0, 10, 10])

nominal_value = np.array([37.060756808501864,2.18663941021782,0.5791356700688765,1.0348449500885029,0.6674105660199474,0.09264433888912414,0.32547145503549796,0.1781240295913104,
                          0.8762308137940877,0.4980495151203558,6,4])
nominal_param = dict()

for i,p in enumerate(names):
    #nominal_param[p] = round(nominal_value[i],1)
    nominal_param[p] = nominal_value[i]
nominal_param

{'A': 37.060756808501864,
 'D': 2.18663941021782,
 'S': 0.5791356700688765,
 'T': 1.0348449500885029,
 'p_hi': 0.6674105660199474,
 'p_lo': 0.09264433888912414,
 'b_prob': 0.32547145503549796,
 'ps': 0.1781240295913104,
 'ews': 0.8762308137940877,
 'lambda': 0.4980495151203558,
 't_r': 6.0,
 'peer_it': 4.0}

In [6]:
np.random.uniform(0.0,100.0,10)

array([48.19796053, 39.19149482, 35.82158129, 60.30352211, 54.47656024,
       49.37297606, 88.72978199, 95.22964201, 58.16631225, 58.66129977])

In [14]:
PER_INTERVAL = 20

all_params = []
suffixes = []

for idx,cur_param_name in enumerate(names):
    cur_range = np.linspace(lower[idx], upper[idx], num=PER_INTERVAL).tolist()
    for val in cur_range:
        cur_param = nominal_param.copy()
        cur_param[cur_param_name] = round(val,1)
        all_params.append(cur_param)
        suffixes.append(cur_param_name)
len(all_params)

240

In [15]:
#first: order the raions
def get_memory(hh):
    if hh<=100000:
        return 8000
    if hh<=300000:
        return 16000
    return 32000

def get_core(hh):
    if hh<=100000:
        return 1
    if hh<=300000:
        return 2
    return 4

In [16]:
base_comb = 40000
for idx, param in enumerate(all_params):
    D = param['D']
    A = param['A']
    T = param['T']
    S = param['S']*100.0
    t_l = int(param['t_r'])
    t_r = t_l

    ps = param['ps']#0.21#random.uniform(0.4,1.0) ##make this a parameter
    ews = param['ews']
    pactive = 1000#random.randint(15,30)
    peer_thresh_hi = param['p_hi']#0.8
    peer_thresh_lo = param['p_lo']
    LAMBDA = param['lambda']
    network_struct = 13
    dynamic_network = 1
    phase_shift = 1000
    use_neighbor = 5#random.sample([2,5,8,10],1)[0]#random.unifrom(0.0,1.0)
    border_cross_prob = param['b_prob']
    multiply_lo = 1.0
    multiply_hi = 1.8

    max_peer_it = int(param['peer_it'])
    hyper_comb = base_comb+idx

    necessary_mem = pd.read_csv('/home/zm8bh/radiation_model/migration_shock/scripts/analysis_notebooks/memory_req_raion.csv')
    raion_df = pd.read_csv('hh_cnts.csv')

    raion_df = raion_df.sort_values(by='hh',ascending=False)
    raion_df['memory'] = raion_df['hh'].apply(lambda x: get_memory(x))
    raion_df['core'] = raion_df['hh'].apply(lambda x: get_core(x))
    raion_df = raion_df.merge(necessary_mem[['Raion','mem_need']],left_on='raion',right_on='Raion',how='inner')
    #raion_df['memory'] = raion_df[["memory", "mem_need"]].max(axis=1)
    raion_df = raion_df.sort_values(by='hh',ascending=False)

    raion = raion_df['raion'].tolist()
    mem = raion_df['memory'].tolist()
    cc = raion_df['core'].tolist()

    partition = 1

    set1_raion = raion[0:partition]
    set1_mem = mem[0:partition]
    set1_cc = cc[0:partition]

    set2_raion = raion[partition:]
    set2_mem = mem[partition:]
    set2_cc = cc[partition:]
    set2_raion.reverse()
    set2_mem.reverse()
    set2_cc.reverse()
    
    f = open("OFAT_SAMPLE_"+suffixes[idx]+"_"+str(hyper_comb)+".sh", "w")
    
    for i in range(len(set1_raion)):
        name = set1_raion[i]
        mem_req = set1_mem[i]
        core_use = set1_cc[i]
        print('sbatch --mem='+str(mem_req)+' --cpus-per-task='+str(core_use)+' abm.sbatch',end=' ',file=f)
        if name.startswith('Chornobyl'):
            print('"'+name+'"',hyper_comb,D,A,T,S,t_l,t_r,ps,ews,pactive,peer_thresh_lo,peer_thresh_hi,
                  LAMBDA,dynamic_network,use_neighbor,border_cross_prob,phase_shift,multiply_lo,multiply_hi,core_use,max_peer_it,file=f)
        else:
            print(name,hyper_comb,D,A,T,S,t_l,t_r,ps,ews,pactive,peer_thresh_lo,peer_thresh_hi,
                  LAMBDA,dynamic_network,use_neighbor,border_cross_prob,phase_shift,multiply_lo,multiply_hi,core_use,max_peer_it,file=f)


    for i in range(len(set2_raion)):
        name = set2_raion[i]
        mem_req = set2_mem[i]
        core_use = set2_cc[i]
        print('sbatch --mem='+str(mem_req)+' --cpus-per-task='+str(core_use)+' abm.sbatch',end=' ',file=f)
        if name.startswith('Chornobyl'):
            print('"'+name+'"',hyper_comb,D,A,T,S,t_l,t_r,ps,ews,pactive,peer_thresh_lo,peer_thresh_hi,
                  LAMBDA,dynamic_network,use_neighbor,border_cross_prob,phase_shift,multiply_lo,multiply_hi,core_use,max_peer_it,file=f)
        else:
            print(name,hyper_comb,D,A,T,S,t_l,t_r,ps,ews,pactive,peer_thresh_lo,peer_thresh_hi,
                  LAMBDA,dynamic_network,use_neighbor,border_cross_prob,phase_shift,multiply_lo,multiply_hi,core_use,max_peer_it,file=f)
    f.close()


In [17]:
base_comb = 40000
for idx, param in enumerate(all_params):
    hyper_comb = base_comb+idx
    filename = "OFAT_SAMPLE_"+suffixes[idx]+"_"+str(hyper_comb)+".sh"
    print('bash',filename)

bash OFAT_SAMPLE_A_40000.sh
bash OFAT_SAMPLE_A_40001.sh
bash OFAT_SAMPLE_A_40002.sh
bash OFAT_SAMPLE_A_40003.sh
bash OFAT_SAMPLE_A_40004.sh
bash OFAT_SAMPLE_A_40005.sh
bash OFAT_SAMPLE_A_40006.sh
bash OFAT_SAMPLE_A_40007.sh
bash OFAT_SAMPLE_A_40008.sh
bash OFAT_SAMPLE_A_40009.sh
bash OFAT_SAMPLE_A_40010.sh
bash OFAT_SAMPLE_A_40011.sh
bash OFAT_SAMPLE_A_40012.sh
bash OFAT_SAMPLE_A_40013.sh
bash OFAT_SAMPLE_A_40014.sh
bash OFAT_SAMPLE_A_40015.sh
bash OFAT_SAMPLE_A_40016.sh
bash OFAT_SAMPLE_A_40017.sh
bash OFAT_SAMPLE_A_40018.sh
bash OFAT_SAMPLE_A_40019.sh
bash OFAT_SAMPLE_D_40020.sh
bash OFAT_SAMPLE_D_40021.sh
bash OFAT_SAMPLE_D_40022.sh
bash OFAT_SAMPLE_D_40023.sh
bash OFAT_SAMPLE_D_40024.sh
bash OFAT_SAMPLE_D_40025.sh
bash OFAT_SAMPLE_D_40026.sh
bash OFAT_SAMPLE_D_40027.sh
bash OFAT_SAMPLE_D_40028.sh
bash OFAT_SAMPLE_D_40029.sh
bash OFAT_SAMPLE_D_40030.sh
bash OFAT_SAMPLE_D_40031.sh
bash OFAT_SAMPLE_D_40032.sh
bash OFAT_SAMPLE_D_40033.sh
bash OFAT_SAMPLE_D_40034.sh
bash OFAT_SAMPLE_D_4

In [13]:
def get_params_from_file(filename):
    with open(filename,'r') as f:
        line = f.readlines()[0]
        args = line.split()
        #print(args)
        param_dict = {'D':float(args[6]),'A':float(args[7]),'T':float(args[8]),'S':float(args[9]),'t_r':int(args[10]),'beta':float(args[12])*float(args[13]),'pi_lo':float(args[15]),
                      'pi_hi':float(args[16]),'lambda_1':float(args[17]),'lambda_2':1-float(args[17]),'b_prob':float(args[20]),'p_iter':int(args[25])}
    f.close()
    return param_dict

for i in range(20000,20100):
    filename = "LHS_SAMPLE_"+str(i)+".sh"
    param_dict = get_params_from_file(filename)
    print(i,param_dict)

20000 {'D': 2.831176387922759, 'A': 45.41181994051273, 'T': 2.4221791687691816, 'S': 46.54190938136089, 't_r': 3, 'beta': 0.19846363364494074, 'pi_lo': 0.0949401527862946, 'pi_hi': 0.4902624564721476, 'lambda_1': 0.7409181929435898, 'lambda_2': 0.25908180705641015, 'b_prob': 0.3350757550581053, 'p_iter': 2}
20001 {'D': 4.435379483905869, 'A': 33.60436135989398, 'T': 2.6341214871499368, 'S': 73.34932148924065, 't_r': 8, 'beta': 0.7095788970812333, 'pi_lo': 0.06604511100265104, 'pi_hi': 0.8828671465781186, 'lambda_1': 0.589244200813538, 'lambda_2': 0.410755799186462, 'b_prob': 0.33121347003631685, 'p_iter': 3}
20002 {'D': 4.7646922572103545, 'A': 69.64808043985195, 'T': 1.0077230632487781, 'S': 9.49549004032902, 't_r': 3, 'beta': 0.17266539692271135, 'pi_lo': 0.027455644158225344, 'pi_hi': 0.8386651808096353, 'lambda_1': 0.7104427315818509, 'lambda_2': 0.2895572684181491, 'b_prob': 0.3561154561100148, 'p_iter': 2}
20003 {'D': 4.187795634898415, 'A': 48.987642960843864, 'T': 1.69542696034

In [12]:

    
get_params_from_file('LHS_SAMPLE_20000.sh')

['sbatch', '--mem=32000', '--cpus-per-task=4', 'abm.sbatch', 'Kyiv', '20000', '2.831176387922759', '45.41181994051273', '2.4221791687691816', '46.54190938136089', '3', '3', '0.6866381280695788', '0.2890367218652762', '1000', '0.0949401527862946', '0.4902624564721476', '0.7409181929435898', '1', '5', '0.3350757550581053', '1000', '1.0', '1.8', '4', '2']
{'D': 2.831176387922759, 'A': 45.41181994051273, 'T': 2.4221791687691816, 'S': 46.54190938136089, 't_r': 3, 'beta': 0.19846363364494074, 'pi_lo': 0.0949401527862946, 'pi_hi': 0.4902624564721476, 'lambda_1': 0.7409181929435898, 'lambda_2': 0.25908180705641015, 'b_prob': 0.3350757550581053, 'p_iter': 2}
